In [6]:
import re
import subprocess
import asyncio

from metagpt.actions import Action
from metagpt.roles.role import Role, RoleReactMode
from metagpt.schema import Message
from metagpt.logs import logger

# 使用现成的智能体

In [2]:
# 可导入任何角色，初始化它，用一个开始的消息运行它，完成！
import asyncio

from metagpt.context import Context
from metagpt.roles.product_manager import ProductManager
from metagpt.logs import logger

async def main():
    msg = "Write a PRD for a snake game"
    context = Context()  # 显式创建会话Context对象，Role对象会隐式的自动将它共享给自己的Action对象
    role = ProductManager(context=context)
    while msg:
        msg = await role.run(msg)
        logger.info(str(msg))

if __name__ == '__main__':
    # asyncio.run(main())
    await main()

2025-03-06 22:00:16.171 | INFO     | metagpt.roles.role:_act:391 - Alice(Product Manager): to do PrepareDocuments(PrepareDocuments)
2025-03-06 22:00:16.474 | INFO     | metagpt.utils.file_repository:save:57 - save to: C:\Users\zata\Desktop\wow\02-metaGPT\workspace\20250306220016\docs\requirement.txt
2025-03-06 22:00:16.475 | INFO     | __main__:main:14 - Alice(Product Manager): {'root_path': 'docs', 'filename': 'requirement.txt', 'content': 'Write a PRD for a snake game'}
2025-03-06 22:00:16.477 | INFO     | metagpt.roles.role:_act:391 - Alice(Product Manager): to do WritePRD(WritePRD)
2025-03-06 22:00:16.481 | INFO     | metagpt.actions.write_prd:run:86 - New requirement detected: Write a PRD for a snake game


[CONTENT]
{
    "Language": "en_us",
    "Programming Language": "Python",
    "Original Requirements": "Write a PRD for a snake game",
    "Project Name": "snake_game",
    "Product Goals": [
        "Develop an intuitive and engaging gameplay experience",
        "Ensure the game is accessible and responsive across devices",
        "Implement a high-quality, visually appealing user interface"
    ],
    "User Stories": [
        "As a player, I want to easily navigate the game controls to play the snake game",
        "As a player, I want to see my score and high scores displayed prominently",
        "As a player, I want the ability to pause and resume the game at any time",
        "As a player, I want to have the option to restart the game from the beginning",
        "As a player, I want the game to be visually appealing with smooth animations"
    ],
    "Competitive Analysis": [
        "Snake Game A: Basic gameplay, lacks advanced features and graphics",
        "Snake Game B

2025-03-06 22:00:37.888 | WARNING  | metagpt.utils.cost_manager:update_cost:49 - Model glm-4-flash not found in TOKEN_COSTS.


."
}
[/CONTENT]


2025-03-06 22:00:37.994 | WARNING  | metagpt.utils.git_repository:rename_root:214 - Move C:\Users\zata\Desktop\wow\02-metaGPT\workspace\20250306220016 to C:\Users\zata\Desktop\wow\02-metaGPT\workspace\snake_game error: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\zata\\Desktop\\wow\\02-metaGPT\\workspace\\20250306220016'
2025-03-06 22:00:37.996 | INFO     | metagpt.utils.git_repository:rename_root:219 - Rename directory C:\Users\zata\Desktop\wow\02-metaGPT\workspace\20250306220016 to C:\Users\zata\Desktop\wow\02-metaGPT\workspace\snake_game
2025-03-06 22:00:38.004 | INFO     | metagpt.utils.file_repository:save:57 - save to: C:\Users\zata\Desktop\wow\02-metaGPT\workspace\snake_game\docs\prd\20250306220038.json
2025-03-06 22:00:38.106 | WARNING  | metagpt.utils.mermaid:mermaid_to_file:35 - RUN `npm install -g @mermaid-js/mermaid-cli` to install mmdc,or consider changing engine to `playwright`, `pyppeteer`, or `ink`.
2025-03-06

# 具有单一动作的智能体

## 定义动作

In [7]:


import re
from metagpt.actions import Action
# Action是动作的逻辑抽象。用户可以通过简单地调用 self._aask 函数令 LLM 赋予这个动作能力，即这个函数将在底层调用 LLM api。
# 因此可以通过from metagpt.actions import Action 来测试config.yaml是否配置成功

class SimpleWriteCode(Action):

    # 使用 PROMPT_TEMPLATE 来定义一个动作，这个动作是一个函数，这个函数会调用 LLM api，并且返回一个字符串。
    PROMPT_TEMPLATE: str = """
    Write a python function that can {instruction} and provide two runnnable test cases.
    Return ```python your_code_here ``` with NO other texts,
    your code:
    """

    name: str = "SimpleWriteCode"

    async def run(self, instruction: str):
        prompt = self.PROMPT_TEMPLATE.format(instruction=instruction)

        rsp = await self._aask(prompt)

        code_text = SimpleWriteCode.parse_code(rsp)

        return code_text

    @staticmethod
    def parse_code(rsp):
        pattern = r"```python(.*)```"
        match = re.search(pattern, rsp, re.DOTALL)
        code_text = match.group(1) if match else rsp
        return code_text

## 定义角色

In [12]:
from metagpt.roles import Role

# Role 类是智能体的逻辑抽象。一个 Role 能执行特定的 Action，拥有记忆、思考并采用各种策略行动。基本上，它充当一个将所有这些组件联系在一起的凝聚实体。目前，让我们只关注一个执行动作的智能体，并看看如何定义一个最简单的 Role。
        

# 创建一个simpleCoder角色
class SimpleCoder(Role):
    
    # 定义角色的名字和描述
    name: str = "Alice"
    profile: str = "SimpleCoder"

    # 初始化
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # 配备对应的动作
        self.set_actions([SimpleWriteCode])


    # 重写_act，其中包含智能体具体行动逻辑。我们写入，我们的智能体将从最新的记忆中获取人类指令，运行配备的动作，MetaGPT将其作为待办事项 (self.rc.todo) 在幕后处理，最后返回一个完整的消息。
    async def _act(self) -> Message:
        logger.info(f"{self._setting}: to do {self.rc.todo}({self.rc.todo.name})")
        todo = self.rc.todo  # todo will be SimpleWriteCode()  

        msg = self.get_memories(k=1)[0]  # find the most recent messages
        code_text = await todo.run(msg.content)
        msg = Message(content=code_text, role=self.profile, cause_by=type(todo))

        return msg


## 运行角色

In [ ]:
import asyncio

from metagpt.context import Context

async def main():
    msg = "write a function that calculates the sum of a list"
    context = Context()
    role = SimpleCoder(context=context)
    logger.info(msg)
    result = await role.run(msg)
    logger.info(result)

await main()


# asyncio.run(main) # if .py file

# 具有多个动作的智能体
我们注意到一个智能体能够执行一个动作，但如果只有这些，实际上我们并不需要一个智能体。通过直接运行动作本身，我们可以得到相同的结果。智能体的力量，或者说Role抽象的惊人之处，在于动作的组合（以及其他组件，比如记忆，但我们将把它们留到后面的部分）。通过连接动作，我们可以构建一个工作流程，使智能体能够完成更复杂的任务。

假设现在我们不仅希望用自然语言编写代码，而且还希望生成的代码立即执行。一个拥有多个动作的智能体可以满足我们的需求。让我们称之为RunnableCoder，一个既写代码又立即运行的Role。我们需要两个Action：SimpleWriteCode 和 SimpleRunCode

## 定义动作

In [21]:
# 定义两个动作，其中SimpleWriteCode 就是复用上面的动作

class SimpleWriteCode(Action):

    # 使用 PROMPT_TEMPLATE 来定义一个动作，这个动作是一个函数，这个函数会调用 LLM api，并且返回一个字符串。
    PROMPT_TEMPLATE: str = """
    Write a python function that can {instruction} and provide two runnnable test cases.
    Return ```python your_code_here ``` with NO other texts,
    your code:
    """

    name: str = "SimpleWriteCode"

    async def run(self, instruction: str):
        prompt = self.PROMPT_TEMPLATE.format(instruction=instruction)

        rsp = await self._aask(prompt)

        code_text = SimpleWriteCode.parse_code(rsp)

        return code_text

    @staticmethod
    def parse_code(rsp):
        pattern = r"```python(.*)```"
        match = re.search(pattern, rsp, re.DOTALL)
        code_text = match.group(1) if match else rsp
        return code_text





class SimpleRunCode(Action):
    name: str = "SimpleRunCode"

    async def run(self, code_text: str):
        
        # 从概念上讲，一个动作可以利用LLM，也可以在没有LLM的情况下运行。在SimpleRunCode的情况下，LLM不涉及其中。我们只需启动一个子进程来运行代码并获取结果。我们希望展示的是，对于动作逻辑的结构，我们没有设定任何限制，用户可以根据需要完全灵活地设计逻辑。
        result = subprocess.run(["python", "-c", code_text], capture_output=True, text=True)
        code_result = result.stdout
        logger.info(f"{code_result=}")
        return code_result

## 定义角色

In [22]:
class RunnableCoder(Role):
    name: str = "Alice"
    profile: str = "RunnableCoder"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        #  初始化动作
        self.set_actions([SimpleWriteCode, SimpleRunCode])
        # 指定每次 Role 会选择哪个 Action。我们将 react_mode 设置为 "by_order"，这意味着 Role 将按照 self.set_actions 中指定的顺序执行其能够执行的 Action
        self._set_react_mode(react_mode="by_order")

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: to do {self.rc.todo}({self.rc.todo.name})")
        # By choosing the Action by order under the hood
        # todo will be first SimpleWriteCode() then SimpleRunCode()
        todo = self.rc.todo

        msg = self.get_memories(k=1)[0]  # find the most k recent messages
        result = await todo.run(msg.content)

        msg = Message(content=result, role=self.profile, cause_by=type(todo))
        self.rc.memory.add(msg)
        return msg

## 运行角色

In [23]:
import asyncio

from metagpt.context import Context

async def main():
    msg = "write a function that calculates the sum of a list"
    context = Context()
    role = RunnableCoder(context=context)
    logger.info(msg)
    result = await role.run(msg)
    logger.info(result)

await main()
# asyncio.run(main) # if .py file, use this line

2025-02-21 00:14:29.668 | INFO     | __main__:main:9 - write a function that calculates the sum of a list
2025-02-21 00:14:29.670 | INFO     | __main__:_act:13 - Alice(RunnableCoder): to do SimpleWriteCode(SimpleWriteCode)


```python
def calculate_sum(numbers):
    return sum(numbers)

# Test cases
def test_case_1():
    assert calculate_sum([1, 2, 3, 4, 5]) == 15

def test_case_2():
    assert calculate_sum([10, -5, 3, 2]) ==

2025-02-21 00:14:32.814 | WARNING  | metagpt.utils.cost_manager:update_cost:49 - Model glm-4-flash not found in TOKEN_COSTS.
2025-02-21 00:14:32.816 | INFO     | __main__:_act:13 - Alice(RunnableCoder): to do SimpleRunCode(SimpleRunCode)


 10
```


2025-02-21 00:14:32.908 | INFO     | __main__:run:42 - code_result=''
2025-02-21 00:14:32.910 | INFO     | __main__:main:11 - RunnableCoder: 
